# Convolutional artificial neural network : Image classification on MNIST dataset
- Test Accuracy 98.92% without data augmentation


In [1]:
# Import modules
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import numpy as np

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Input, Activation, Add, Dense, Flatten

### Data preparation

#### Import data

In [2]:
def load_data():
    # Import MNIST dataset from openml
    dataset = fetch_openml('mnist_784', version=1, data_home=None)

    # Data preparation
    raw_X = dataset['data']
    raw_Y = dataset['target']
    return raw_X, raw_Y

raw_X, raw_Y = load_data()

#### Preprocessing

In [3]:
def clean_data(raw_X, raw_Y):

    cleaned_X = raw_X.astype('float32')
    cleaned_X /= 255
    cleaned_X = np.array(cleaned_X)

    cleaned_X = cleaned_X.reshape(cleaned_X.shape[0], 28, 28, 1)
        
    num_classes = 10
    cleaned_Y = keras.utils.to_categorical(raw_Y, num_classes)
    
    return cleaned_X, cleaned_Y
    
cleaned_X, cleaned_Y = clean_data(raw_X, raw_Y)

In [4]:
cleaned_X.shape

(70000, 28, 28, 1)

In [5]:
cleaned_Y.shape

(70000, 10)

#### Data split

- Split your data into a train set (50%), validation set (20%) and a test set (30%).

In [17]:
def split_data(cleaned_X, cleaned_Y):

    X_train, X_test, Y_train, Y_test = train_test_split(cleaned_X, cleaned_Y, test_size=0.3, random_state=42)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=2/7, random_state=42)
    
    return X_val, X_test, X_train, Y_val, Y_test, Y_train

X_val, X_test, X_train, Y_val, Y_test, Y_train = split_data(cleaned_X, cleaned_Y)

In [18]:
X_train.shape

(35000, 28, 28, 1)

In [8]:
X_val.shape

(14000, 28, 28, 1)

In [9]:
X_test.shape

(21000, 28, 28, 1)

### Model

#### Convolutional neural network structure


In [14]:
def build_model():

    model = Sequential()
    
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.20))

    model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
    model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2, 2)))
    model.add(Conv2D(256, kernel_size = (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.summary()
    
    return model

def compile_model(model):

    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

    return model


def train_model(model, X_train, Y_train, X_val, Y_val):

    history = model.fit(X_train, Y_train, batch_size = 128, epochs = 15, verbose=1, validation_data=(X_val, Y_val))

    return model, history

In [12]:
model = build_model()
model = compile_model(model)
model, history = train_model(model, X_train, Y_train, X_val, Y_val)
test_loss, test_accuracy = eval_model(model, X_test, Y_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 128)         147584    
                                                        

In [16]:
def eval_model(model, X_test, Y_test):

    score = model.evaluate(X_test, Y_test, verbose=1)
    test_loss = score[0] 
    test_accuracy = score[1]
    print('Test Loss:', '%.4f' % test_loss)
    print('Test Accuracy:', '%.4f' % test_accuracy)

    return test_loss, test_accuracy

test_loss, test_accuracy = eval_model(model, X_test, Y_test)

657/657 [==============================] - 4s 7ms/step - loss: 0.0457 - accuracy: 0.9892
Test Loss: 0.0457
Test Accuracy: 0.9892
